In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

### Methods

The dataset used in this study was generated from the NOMAD dataset. 6 Rrs bands were used at 6 wavelength corresponding to SeaWiFS spectral coverage; 412, 443, 490, 510, 555, 670 nm. The following data transformation steps were implemented:
* invalid data points were flagged and dropped
* for band ratio algorithms, relevant ratios were pre-computed
* for band ratio algorithms with max(Blue) formulation blue band used in the ratio was identified and flagged
* for PCA-based algorithms, principal components were computed for all bands.
* chlorophyll measurement method was identified and flagged
* all predictors were standardized

In [2]:
def read_nomad_like_file_to_pd(file_name):
    with open(file_name) as f:
        for i, line in enumerate(f.readlines()):
            if 'fields' in line:
                columns = line.split('=')[1].split(',')
                continue
            if '/end_header' in line:
                skiprows = i+1
        df = pd.read_csv(nomad_file, names=columns,
                             skiprows=skiprows, na_values=-999)
    return df

In [3]:
nomad_file = './data/raw/nomad_seabass_v2.a_2008200.txt'
df_raw = read_nomad_like_file_to_pd(nomad_file)
df_raw.columns.tolist()

['year',
 'month',
 'day',
 'hour',
 'minute',
 'second',
 'lat',
 'lon',
 'id',
 'oisst',
 'etopo2',
 'chl',
 'chl_a',
 'kd405',
 'kd411',
 'kd443',
 'kd455',
 'kd465',
 'kd489',
 'kd510',
 'kd520',
 'kd530',
 'kd550',
 'kd555',
 'kd560',
 'kd565',
 'kd570',
 'kd590',
 'kd619',
 'kd625',
 'kd665',
 'kd670',
 'kd683',
 'lw405',
 'lw411',
 'lw443',
 'lw455',
 'lw465',
 'lw489',
 'lw510',
 'lw520',
 'lw530',
 'lw550',
 'lw555',
 'lw560',
 'lw565',
 'lw570',
 'lw590',
 'lw619',
 'lw625',
 'lw665',
 'lw670',
 'lw683',
 'es405',
 'es411',
 'es443',
 'es455',
 'es465',
 'es489',
 'es510',
 'es520',
 'es530',
 'es550',
 'es555',
 'es560',
 'es565',
 'es570',
 'es590',
 'es619',
 'es625',
 'es665',
 'es670',
 'es683',
 'ap405',
 'ap411',
 'ap443',
 'ap455',
 'ap465',
 'ap489',
 'ap510',
 'ap520',
 'ap530',
 'ap550',
 'ap555',
 'ap560',
 'ap565',
 'ap570',
 'ap590',
 'ap619',
 'ap625',
 'ap665',
 'ap670',
 'ap683',
 'ad405',
 'ad411',
 'ad443',
 'ad455',
 'ad465',
 'ad489',
 'ad510',
 'ad520',


In [4]:
bands = [411, 443, 489, 510, 555, 670]

In [5]:
df_raw.loc[:, [f'lw{band}' for band in bands]].describe()

,lw411,lw443,lw489,lw510,lw555,lw670
count,4304.000000,4456.000000,4431.000000,3492.000000,3312.000000,1602.000000
mean,0.385876,0.433970,0.450727,0.416742,0.330696,0.136143
std,0.418701,0.437093,0.390907,0.368770,0.430900,0.231025
min,0.001019,0.001470,0.001470,0.001250,0.000600,0.000000
25%,0.099433,0.118075,0.150520,0.188368,0.120175,0.016921
50%,0.235650,0.280952,0.347400,0.347605,0.210395,0.055817
75%,0.501023,0.609602,0.657315,0.524745,0.330312,0.159858
max,3.525270,2.988000,3.592700,4.398700,6.288500,3.222200


In [6]:
df_raw.loc[:, [f'es{band}' for band in bands]].describe()

,es411,es443,es489,es510,es555,es670
count,4305.000000,4459.000000,4450.000000,3446.000000,3258.000000,1616.000000
mean,73.540755,85.777988,93.053646,99.686608,97.828945,96.691502
std,42.527826,49.526399,52.812348,47.437513,45.437147,37.233330
min,0.412000,0.489000,0.479000,0.420000,0.284000,1.287880
25%,34.741000,40.442000,44.969450,61.575050,62.379225,74.092875
50%,75.080000,88.201800,96.760450,105.335000,104.924000,102.500000
75%,110.525000,127.631500,138.594000,139.984750,136.329000,123.000000
max,216.678000,238.739000,249.221000,252.287000,243.629000,194.600000


### $\Rightarrow$ Computing Rrs and creating dataset and storing it in a dataframe (df) for this study:

In [8]:
for band in bands:
    df_raw[f'rrs{band}'] = df_raw[f'lw{band}'] / df_raw[f'es{band}'] # create rrs columns
columns = df_raw.filter(regex='rrs', axis=1).columns # get rrs column names
columns = ['id'] + ['etopo2', 'lat'] + columns.tolist() + ['chl', 'chl_a'] # add features of interest
df = df_raw[columns].copy() # subset and copy
df.head()

,id,etopo2,lat,rrs411,rrs443,rrs489,rrs510,rrs555,rrs670,chl,chl_a
0,1565,0.0,38.4279,0.001204,0.001686,0.003293,0.004036,0.007479,0.003465,38.19,NaN
1,1566,0.0,38.3680,0.001062,0.001384,0.002173,0.002499,0.004152,0.001695,35.01,NaN
2,1567,1.0,38.3074,0.000971,0.001185,0.001843,0.002288,0.004246,0.001612,26.91,NaN
3,1568,3.0,38.6367,0.001472,0.001741,0.002877,0.003664,0.006982,0.003234,47.96,NaN
4,1559,1.0,38.3047,0.000905,0.001022,0.001506,0.001903,0.002801,0.001791,23.55,NaN


In [9]:
df.describe()

,id,etopo2,lat,rrs411,rrs443,rrs489,rrs510,rrs555,rrs670,chl,chl_a
count,4459.000000,4459.000000,4459.000000,4293.000000,4456.000000,4422.000000,3435.000000,3255.000000,1598.000000,3392.000000,1381.000000
mean,4377.381251,1312.346715,1.868658,0.004881,0.004652,0.004590,0.004130,0.003256,0.001557,2.703251,2.285293
std,2298.272102,1766.435289,44.765125,0.003447,0.003002,0.002768,0.003130,0.003536,0.002387,5.611762,5.752391
min,6.000000,0.000000,-77.035600,0.000051,0.000190,0.000284,0.000261,0.000183,0.000000,0.012000,0.017000
25%,2028.500000,18.000000,-61.299000,0.002509,0.002617,0.003051,0.002831,0.001588,0.000200,0.274000,0.145000
50%,5039.000000,240.000000,27.093000,0.003984,0.003899,0.004153,0.003425,0.002071,0.000614,0.830000,0.538000
75%,6271.500000,2789.500000,34.458500,0.006301,0.006076,0.005655,0.004242,0.003141,0.002000,2.240080,1.694000
max,7831.000000,7978.000000,79.690000,0.030600,0.036769,0.063814,0.077740,0.046600,0.027700,77.864800,70.213300


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Data columns (total 11 columns):
id        4459 non-null int64
etopo2    4459 non-null float64
lat       4459 non-null float64
rrs411    4293 non-null float64
rrs443    4456 non-null float64
rrs489    4422 non-null float64
rrs510    3435 non-null float64
rrs555    3255 non-null float64
rrs670    1598 non-null float64
chl       3392 non-null float64
chl_a     1381 non-null float64
dtypes: float64(10), int64(1)
memory usage: 383.3 KB


### $\Rightarrow$ Add hplc flag column to identify protential quality/noise difference in chl

In [11]:
#chlorophyll measurement method was identified and flagged
def fill_chl(row):
    return row['chl_a'] if np.isfinite(row['chl_a']) else row['chl']

df['chlor_a'] = df.apply(fill_chl, axis=1)
df['is_hplc'] = np.isfinite(df.chl_a)
df.drop(['chl', 'chl_a'], axis=1, inplace=True)

In [12]:
df.describe()

,id,etopo2,lat,rrs411,rrs443,rrs489,rrs510,rrs555,rrs670,chlor_a
count,4459.000000,4459.000000,4459.000000,4293.000000,4456.000000,4422.000000,3435.000000,3255.000000,1598.000000,4127.000000
mean,4377.381251,1312.346715,1.868658,0.004881,0.004652,0.004590,0.004130,0.003256,0.001557,2.680228
std,2298.272102,1766.435289,44.765125,0.003447,0.003002,0.002768,0.003130,0.003536,0.002387,5.758436
min,6.000000,0.000000,-77.035600,0.000051,0.000190,0.000284,0.000261,0.000183,0.000000,0.012000
25%,2028.500000,18.000000,-61.299000,0.002509,0.002617,0.003051,0.002831,0.001588,0.000200,0.233325
50%,5039.000000,240.000000,27.093000,0.003984,0.003899,0.004153,0.003425,0.002071,0.000614,0.764000
75%,6271.500000,2789.500000,34.458500,0.006301,0.006076,0.005655,0.004242,0.003141,0.002000,2.150000
max,7831.000000,7978.000000,79.690000,0.030600,0.036769,0.063814,0.077740,0.046600,0.027700,77.864800


In [13]:
df.head()

,id,etopo2,lat,rrs411,rrs443,rrs489,rrs510,rrs555,rrs670,chlor_a,is_hplc
0,1565,0.0,38.4279,0.001204,0.001686,0.003293,0.004036,0.007479,0.003465,38.19,False
1,1566,0.0,38.3680,0.001062,0.001384,0.002173,0.002499,0.004152,0.001695,35.01,False
2,1567,1.0,38.3074,0.000971,0.001185,0.001843,0.002288,0.004246,0.001612,26.91,False
3,1568,3.0,38.6367,0.001472,0.001741,0.002877,0.003664,0.006982,0.003234,47.96,False
4,1559,1.0,38.3047,0.000905,0.001022,0.001506,0.001903,0.002801,0.001791,23.55,False


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Data columns (total 11 columns):
id         4459 non-null int64
etopo2     4459 non-null float64
lat        4459 non-null float64
rrs411     4293 non-null float64
rrs443     4456 non-null float64
rrs489     4422 non-null float64
rrs510     3435 non-null float64
rrs555     3255 non-null float64
rrs670     1598 non-null float64
chlor_a    4127 non-null float64
is_hplc    4459 non-null bool
dtypes: bool(1), float64(9), int64(1)
memory usage: 352.8 KB


In [15]:
df.to_pickle('./pickleJar/df_main.pkl') # pickle nomad subset
df_raw.to_pickle('./pickleJar/df_raw.pkl') # pickle nomad raw data in dataframe

### End of this notebook on dataset cleanup & prep. 
### Next: BC-0-Cleanup-1 $\Rightarrow$ Subset the prepared dataset for specific studies.